In [1]:
from ultralytics import YOLO
model = YOLO("../../training-jobs/detect/cricket-object-detect17/weights/best.pt")  # load a custom trained
# Export the model
model.export(format='onnx')

Ultralytics YOLOv8.0.79 🚀 Python-3.10.6 torch-2.0.0+cu117 CPU
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from ../../training-jobs/detect/cricket-object-detect17/weights/best.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (23.4 MB)

ONNX: starting export with onnx 1.13.1 opset 17...
ONNX: export success ✅ 0.5s, saved as ../../training-jobs/detect/cricket-object-detect17/weights/best.onnx (11.7 MB)

Export complete (1.0s)
Results saved to /home/osurasenith/Projects/third-umpire-decision-automation/backend/training-jobs/detect/cricket-object-detect17/weights
Predict:         yolo predict task=detect model=../../training-jobs/detect/cricket-object-detect17/weights/best.onnx imgsz=640 
Validate:        yolo val task=detect model=../../training-jobs/detect/cricket-object-detect17/weights/best.onnx imgsz=640 data=./datasets/cricket-object-detect-augmented/cricket-object-detect-augmented.yaml 
Visualize:       h

============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



'../../training-jobs/detect/cricket-object-detect17/weights/best.onnx'